# Overview

The purpose of this project was to experiment with different ways to query a database containing a large volume of CitiBike trip data. The data was preloaded into CartoDB and queried via SQL.

## Setup

In [1]:
try:
    import urllib2 as ulib
except ImportError:
    import urllib3 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd
from geopy.distance import great_circle

### Query functions

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

In [3]:
# Get all the records and sort by start_station_id, then by tripduration (for trips <= 3 hrs)
query = '''SELECT * FROM fb55.citibike
WHERE tripduration <= (3*60*60)
ORDER BY start_station_id DESC, tripduration DESC;
'''
table1 = get_data(query)
table1.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,South End Ave & Liberty St,NaN,1576,-74.015756,7225,2015-02-01 11:46:00+00,2015-02-01 13:47:00+00,3002,1506,40.711512,499,Broadway & W 60 St,40.769155,-73.981918,21202,Customer,NaN,0
1,NaN,South End Ave & Liberty St,NaN,8891,-74.015756,2139,2015-02-03 09:33:00+00,2015-02-03 10:09:00+00,3002,8831,40.711512,510,W 51 St & 6 Ave,40.760660,-73.980420,15552,Subscriber,1958.0,1
2,NaN,South End Ave & Liberty St,NaN,23568,-74.015756,2131,2015-02-05 08:42:00+00,2015-02-05 09:17:00+00,3002,23529,40.711512,359,E 47 St & Park Ave,40.755103,-73.974987,21473,Subscriber,1985.0,1
3,NaN,South End Ave & Liberty St,NaN,27243,-74.015756,1977,2015-02-05 16:11:00+00,2015-02-05 16:44:00+00,3002,27203,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,20069,Subscriber,1969.0,1
4,NaN,South End Ave & Liberty St,NaN,10232,-74.015756,1977,2015-02-03 16:04:00+00,2015-02-03 16:37:00+00,3002,10173,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,15309,Subscriber,1969.0,1
5,NaN,South End Ave & Liberty St,NaN,6846,-74.015756,1976,2015-02-02 09:21:00+00,2015-02-02 09:53:00+00,3002,6781,40.711512,494,W 26 St & 8 Ave,40.747348,-73.997236,19665,Subscriber,1959.0,1
6,NaN,South End Ave & Liberty St,NaN,44708,-74.015756,1920,2015-02-07 18:36:00+00,2015-02-07 19:08:00+00,3002,44703,40.711512,410,Suffolk St & Stanton St,40.720664,-73.985180,15443,Subscriber,1964.0,2
7,NaN,South End Ave & Liberty St,NaN,11648,-74.015756,1908,2015-02-03 18:54:00+00,2015-02-03 19:26:00+00,3002,11587,40.711512,279,Peck Slip & Front St,40.707873,-74.001670,19133,Subscriber,1996.0,1
8,NaN,South End Ave & Liberty St,NaN,1297,-74.015756,1900,2015-02-01 11:14:00+00,2015-02-01 11:45:00+00,3002,1228,40.711512,525,W 34 St & 11 Ave,40.755942,-74.002116,17849,Customer,NaN,0
9,NaN,South End Ave & Liberty St,NaN,17637,-74.015756,1830,2015-02-04 16:33:00+00,2015-02-04 17:04:00+00,3002,17590,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,20246,Subscriber,1969.0,1


In [4]:
# Select head of data
query = '''
SELECT * FROM fb55.citibike
'''
table1 = get_data(query)
table1.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0
5,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
6,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
7,NaN,E 39 St & 3 Ave,NaN,8327,-73.976049,441,2015-02-03 08:02:00+00,2015-02-03 08:10:00+00,167,8263,40.748901,526,E 33 St & 5 Ave,40.747659,-73.984907,21518,Subscriber,1975.0,1
8,NaN,10 Ave & W 28 St,NaN,9435,-74.001768,1475,2015-02-03 12:31:00+00,2015-02-03 12:56:00+00,489,9375,40.750664,455,1 Ave & E 44 St,40.750020,-73.969053,19863,Subscriber,1984.0,2
9,NaN,E 2 St & 2 Ave,NaN,10472,-73.990697,420,2015-02-03 16:45:00+00,2015-02-03 16:52:00+00,403,10411,40.725029,297,E 15 St & 3 Ave,40.734232,-73.986923,19257,Subscriber,1984.0,1


In [5]:
# Select tail of data
query = '''
SELECT * FROM fb55.citibike
'''
table1 = get_data(query)
table1.tail(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
46190,NaN,W 17 St & 8 Ave,NaN,46190,-74.001497,306,2015-02-07 23:55:00+00,2015-02-08 00:01:00+00,116,46191,40.741776,494,W 26 St & 8 Ave,40.747348,-73.997236,14736,Subscriber,1983.0,2
46191,NaN,W 47 St & 10 Ave,NaN,46191,-73.993012,1248,2015-02-07 23:56:00+00,2015-02-08 00:17:00+00,495,46192,40.762699,432,E 7 St & Avenue A,40.726218,-73.983799,16300,Subscriber,1984.0,1
46192,NaN,Warren St & Church St,NaN,46192,-74.009106,1165,2015-02-07 23:56:00+00,2015-02-08 00:15:00+00,152,46193,40.714740,325,E 19 St & 3 Ave,40.736245,-73.984738,16978,Subscriber,1959.0,1
46193,NaN,E 20 St & 2 Ave,NaN,46193,-73.982050,993,2015-02-07 23:56:00+00,2015-02-08 00:13:00+00,461,46194,40.735877,295,Pike St & E Broadway,40.714067,-73.992939,16722,Subscriber,1974.0,1
46194,NaN,1 Ave & E 44 St,NaN,46194,-73.969053,1422,2015-02-07 23:57:00+00,2015-02-08 00:20:00+00,455,46195,40.750020,265,Stanton St & Chrystie St,40.722293,-73.991475,20184,Subscriber,1960.0,2
46195,NaN,W 20 St & 8 Ave,NaN,46195,-74.000040,689,2015-02-07 23:57:00+00,2015-02-08 00:08:00+00,470,46196,40.743453,325,E 19 St & 3 Ave,40.736245,-73.984738,15585,Subscriber,1953.0,1
46196,NaN,5 Ave & E 29 St,NaN,46196,-73.986831,428,2015-02-07 23:57:00+00,2015-02-08 00:04:00+00,474,46197,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,16395,Subscriber,1986.0,2
46197,NaN,5 Ave & E 29 St,NaN,46197,-73.986831,392,2015-02-07 23:57:00+00,2015-02-08 00:03:00+00,474,46198,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,15545,Subscriber,1986.0,1
46198,NaN,Carmine St & 6 Ave,NaN,46198,-74.002150,548,2015-02-07 23:58:00+00,2015-02-08 00:08:00+00,368,46199,40.730386,334,W 20 St & 7 Ave,40.742388,-73.997262,19540,Subscriber,1983.0,2
46199,NaN,E 4 St & 2 Ave,NaN,46199,-73.989780,917,2015-02-07 23:59:00+00,2015-02-08 00:15:00+00,439,46200,40.726281,417,Barclay St & Church St,40.712912,-74.010202,20998,Subscriber,1965.0,2


In [6]:
# Select all unique start_station_id numbers
query = '''
SELECT DISTINCT start_station_id FROM fb55.citibike
ORDER BY start_station_id ASC
'''
table4 = get_data(query)
table4

,start_station_id,Unnamed: 1
0,72,NaN
1,79,NaN
2,82,NaN
3,83,NaN
4,116,NaN
5,119,NaN
6,120,NaN
7,127,NaN
8,128,NaN
9,137,NaN


In [7]:
# Count total trips
query = '''
SELECT COUNT(cartodb_id)
FROM fb55.citibike
'''

table5 = get_data(query)
table5

,count,Unnamed: 1
0,46200,NaN


In [8]:
# Select max min avg trip duration

query = '''
SELECT MAX(tripduration) AS trip_dur_max, 
MIN(tripduration) AS trip_dur_min, 
AVG(tripduration) AS trip_dur_avg
FROM fb55.citibike
'''
table6 = get_data(query)
table6

,trip_dur_max,trip_dur_min,trip_dur_avg
0,43016,60,675.865823


In [9]:
# Select all the trips that started on 2-2-15
query = '''
SELECT * FROM fb55.citibike
WHERE starttime::date = '2015-02-02'
'''
table7 = get_data(query)
table7.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2


In [10]:
# Get all the weekend trips
query = '''
SELECT *, EXTRACT(dow FROM starttime::date) AS dow
FROM fb55.citibike
WHERE EXTRACT(dow FROM starttime::date) IN (0,6);
'''
table9 = get_data(query)
table9

,the_geom,start_station_latitude,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,start_station_name,cartodb_id,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,dow
0,NaN,40.738274,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,E 20 St & Park Ave,107,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1,0
1,NaN,40.760094,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,W 43 St & 10 Ave,1088,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1,0
2,NaN,40.724537,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,E 6 St & Avenue B,2759,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1,0
3,NaN,40.732233,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,E 12 St & 3 Ave,4893,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1,0
4,NaN,40.756405,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,W 41 St & 8 Ave,6090,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0,0
5,NaN,40.735877,NaN,39894,-73.982050,296,2015-02-07 08:13:00+00,2015-02-07 08:18:00+00,461,E 20 St & 2 Ave,39876,435,W 21 St & 6 Ave,40.741740,-73.994156,17278,Subscriber,1987.0,1,6
6,NaN,40.769155,NaN,41512,-73.981918,600,2015-02-07 12:35:00+00,2015-02-07 12:45:00+00,499,Broadway & W 60 St,41501,523,W 38 St & 8 Ave,40.754666,-73.991382,15224,Subscriber,1959.0,1,6
7,NaN,40.722281,NaN,41595,-73.976687,1203,2015-02-07 12:47:00+00,2015-02-07 13:07:00+00,411,E 6 St & Avenue D,41585,428,E 3 St & 1 Ave,40.724677,-73.987834,21461,Subscriber,1968.0,1,6
8,NaN,40.727434,NaN,41806,-73.993790,520,2015-02-07 13:16:00+00,2015-02-07 13:25:00+00,229,Great Jones St,41796,168,W 18 St & 6 Ave,40.739713,-73.994564,14621,Subscriber,1964.0,1,6
9,NaN,40.736494,NaN,42316,-73.997044,487,2015-02-07 14:18:00+00,2015-02-07 14:26:00+00,345,W 13 St & 6 Ave,42307,509,9 Ave & W 22 St,40.745497,-74.001971,21362,Subscriber,1962.0,1,6


In [11]:
# Average weekend trip duration
query = '''
SELECT avg(tripduration) AS avg_weekend_duration
FROM fb55.citibike
WHERE EXTRACT(dow FROM starttime::date) IN (0,6);
'''
table10 = get_data(query)
table10.head(5)

,avg_weekend_duration,Unnamed: 1
0,662.942181,NaN


In [12]:
# Get all the weekday trips
query = '''
SELECT *, EXTRACT(dow FROM starttime::date) AS dow
FROM fb55.citibike
WHERE EXTRACT(dow FROM starttime::date) BETWEEN 1 AND 5;
'''
table11 = get_data(query)
table11.head(5)

,the_geom,start_station_latitude,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,start_station_name,cartodb_id,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,dow
0,NaN,40.745168,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,5 Ave & E 29 St,7367,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1,1
1,NaN,40.716059,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,Allen St & Hester St,7712,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0,1
2,NaN,40.748901,NaN,8327,-73.976049,441,2015-02-03 08:02:00+00,2015-02-03 08:10:00+00,167,E 39 St & 3 Ave,8263,526,E 33 St & 5 Ave,40.747659,-73.984907,21518,Subscriber,1975.0,1,2
3,NaN,40.750664,NaN,9435,-74.001768,1475,2015-02-03 12:31:00+00,2015-02-03 12:56:00+00,489,10 Ave & W 28 St,9375,455,1 Ave & E 44 St,40.750020,-73.969053,19863,Subscriber,1984.0,2,2
4,NaN,40.725029,NaN,10472,-73.990697,420,2015-02-03 16:45:00+00,2015-02-03 16:52:00+00,403,E 2 St & 2 Ave,10411,297,E 15 St & 3 Ave,40.734232,-73.986923,19257,Subscriber,1984.0,1,2


In [13]:
# Get the weekday trip duration
query = '''
SELECT avg(tripduration) AS avg_weekday_duration
FROM fb55.citibike
WHERE EXTRACT(dow FROM starttime::date) BETWEEN 1 AND 5;
'''
table12 = get_data(query)
table12.head(5)

,avg_weekday_duration,Unnamed: 1
0,681.052292,NaN


In [14]:
# Group by station location
query = '''
SELECT start_station_name
FROM fb55.citibike
GROUP BY start_station_name
ORDER BY start_station_name;
'''
table13 = get_data(query)
table13

,start_station_name,Unnamed: 1
0,10 Ave & W 28 St,NaN
1,11 Ave & W 27 St,NaN
2,11 Ave & W 41 St,NaN
3,11 Ave & W 59 St,NaN
4,12 Ave & W 40 St,NaN
5,1 Ave & E 15 St,NaN
6,1 Ave & E 44 St,NaN
7,2 Ave & E 31 St,NaN
8,2 Ave & E 58 St,NaN
9,3 Ave & Schermerhorn St,NaN


In [15]:
# Number of trips per station
query = '''
SELECT start_station_name, COUNT(cartodb_id)
FROM fb55.citibike
GROUP BY start_station_name
ORDER BY start_station_name;
'''
table14 = get_data(query)
table14

,start_station_name,count
0,10 Ave & W 28 St,203
1,11 Ave & W 27 St,172
2,11 Ave & W 41 St,229
3,11 Ave & W 59 St,120
4,12 Ave & W 40 St,91
5,1 Ave & E 15 St,397
6,1 Ave & E 44 St,110
7,2 Ave & E 31 St,302
8,2 Ave & E 58 St,64
9,3 Ave & Schermerhorn St,19


In [16]:
# Number of trips per starting station w/in 500 meters of Times Square
query = '''
SELECT distcalc.start_station_name, COUNT(distcalc.cartodb_id)
FROM (SELECT *, 
    ST_Distance(
    CDB_LatLng(start_station_latitude,start_station_longitude)::geography,
    CDB_LatLng(40.7577,-73.9857)::geography, True) AS dist_from_tsq
    FROM fb55.citibike) AS distcalc
WHERE distcalc.dist_from_tsq <= 500
GROUP BY distcalc.start_station_name
ORDER BY distcalc.start_station_name
'''
table15 = get_data(query)
table15

,start_station_name,count
0,Broadway & W 41 St,251
1,Broadway & W 49 St,213
2,W 41 St & 8 Ave,507
3,W 42 St & 8 Ave,221
4,W 43 St & 6 Ave,112
5,W 45 St & 6 Ave,141
6,W 45 St & 8 Ave,141


In [17]:
# Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare
query = '''
SELECT distcalc.start_station_name, AVG(distcalc.tripduration) AS avg_dur
FROM (SELECT *, 
    ST_Distance(
    CDB_LatLng(start_station_latitude,start_station_longitude)::geography,
    CDB_LatLng(40.7577,-73.9857)::geography, True) AS dist_from_tsq
    FROM fb55.citibike) AS distcalc
WHERE (distcalc.dist_from_tsq <= 500) AND (EXTRACT(dow FROM distcalc.starttime::date) IN (0,6))
GROUP BY distcalc.start_station_name
ORDER BY avg_dur DESC
'''
table16 = get_data(query)
table16.head(1)

,start_station_name,avg_dur
0,Broadway & W 49 St,1010.104167
